In [1]:
!pip install datasets -q

In [2]:
!pip install evaluate -q

In [3]:
!pip install transformers[torch] -q

In [4]:
!pip install accelerate==0.27.2 -U -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.0 MB/s eta 0:00:00


In [28]:
import pandas as pd
import torch
from sklearn.metrics import (
    f1_score,
    recall_score,
    precision_score,
    classification_report,
)
from datasets import load_dataset
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from torch.utils.data import DataLoader
import torch
from torch import nn
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [29]:

df = pd.read_csv("data/data.csv")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class_weights = (1 - (df["labels"].value_counts().sort_index() / len(df))).values
class_weights = torch.from_numpy(class_weights).float().to(device)


class_weights


tensor([0.9921, 0.9891, 0.9852, 0.9842, 0.9871, 0.9891, 0.9486, 0.8823, 0.9980,
        0.9980, 0.9980, 0.9862, 0.9891, 0.9832, 0.9871, 0.9862, 0.9852, 0.9951,
        0.9960, 0.9990, 0.9881, 0.9891, 0.9871, 0.9871, 0.9802, 0.9723, 0.9881,
        0.9871, 0.9941, 0.9891, 0.9862, 0.9832, 0.9881, 0.9871, 0.9832, 0.9842,
        0.9842, 0.9842, 0.9842, 0.9842, 0.9862, 0.9842, 0.9921, 0.9891, 0.9911,
        0.9881, 0.9891, 0.9743, 0.9871, 0.9990, 0.9941, 0.9881, 0.9911, 0.9921,
        0.9921, 0.9921, 0.9871, 0.9862, 0.9852, 0.9911, 0.9911, 0.9881],
       device='cuda:0')

In [44]:
from datasets import load_dataset, Dataset, ClassLabel
import pandas as pd

def get_dataset(csv_path, test_size=0.35, min_samples_per_class=2):
    # Загрузка данных
    full_dataset = load_dataset("csv", data_files=csv_path)["train"]

    # Фильтрация пустых значений
    full_dataset = full_dataset.filter(
        lambda example: example["text"] is not None and example["labels"] is not None
    )

    # Подсчет количества образцов для каждого класса
    df = full_dataset.to_pandas()
    class_counts = df['labels'].value_counts()

    # Исключение классов с недостаточным количеством образцов
    valid_classes = class_counts[class_counts >= min_samples_per_class].index.tolist()
    filtered_df = df[df['labels'].isin(valid_classes)]

    # Преобразование столбца labels в ClassLabel
    unique_labels = filtered_df['labels'].unique().tolist()
    class_label = ClassLabel(num_classes=len(unique_labels), names=unique_labels)

    # Преобразование столбца labels в числовые значения
    filtered_df['labels'] = filtered_df['labels'].apply(lambda x: class_label.str2int(x))

    # Создание нового набора данных
    filtered_dataset = Dataset.from_pandas(filtered_df)

    # Обновление информации о характеристиках набора данных
    filtered_dataset = filtered_dataset.cast_column("labels", class_label)

    # Разделение данных на обучающую и тестовую выборки
    dataset = filtered_dataset.train_test_split(test_size=test_size, stratify_by_column="labels")

    return dataset


In [64]:
dataset = get_dataset("data/data.csv")
labels = sorted(df["labels"].value_counts().keys())

id2label = {}
label2id = {}
for i, label in enumerate(labels):
    id2label[i] = label
    label2id[label] = i

model_name = "DeepPavlov/rubert-base-cased"


<ipython-input-44-9ec8e7ac9763>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['labels'] = filtered_df['labels'].apply(lambda x: class_label.str2int(x))


Casting the dataset:   0%|          | 0/879 [00:00<?, ? examples/s]

In [46]:
dataset

DatasetDict({
    train: Dataset({
        features: ['№ услуги', 'Полное наименование услуги', 'Сокращенное наименование услуги', 'labels', 'text', 'Ответ', '__index_level_0__'],
        num_rows: 571
    })
    test: Dataset({
        features: ['№ услуги', 'Полное наименование услуги', 'Сокращенное наименование услуги', 'labels', 'text', 'Ответ', '__index_level_0__'],
        num_rows: 308
    })
})

In [65]:
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=512)


In [48]:

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)


In [49]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/571 [00:00<?, ? examples/s]

Map:   0%|          | 0/308 [00:00<?, ? examples/s]

In [66]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(labels), id2label=id2label, label2id=label2id
)

model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [51]:
from transformers import AdamW, get_scheduler

dataset_len = dataset["train"].num_rows + dataset["test"].num_rows

optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 5
num_training_steps = num_epochs * dataset_len

lr_scheduler = get_scheduler(
    "cosine",
    optimizer=optimizer,
    num_warmup_steps=int(0.1 * num_epochs * dataset_len),
    num_training_steps=num_training_steps,
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [52]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")

        outputs = model(**inputs)
        logits = outputs.get("logits")

        loss_fct = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

In [53]:
f1_metric = evaluate.load("f1")

training_args = TrainingArguments(
    output_dir="./results/multiclass-rubert/",
    num_train_epochs=50,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    warmup_steps=200,
    weight_decay=0.01,
    logging_strategy="no",
    evaluation_strategy="steps",
    eval_steps=50,
    save_only_model=True,
    save_strategy="steps",
    save_steps=50,
    save_total_limit=2,
    metric_for_best_model="f1",
    greater_is_better=True,
    eval_accumulation_steps=32,
    fp16=True,  # mixed precision
    load_best_model_at_end=True,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [54]:


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return f1_metric.compute(
        predictions=predictions, references=labels, average="macro"
    )


In [55]:


trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
    optimizers=[optimizer, lr_scheduler],
)

trainer.train()



Step,Training Loss,Validation Loss,F1
50,No log,4.024919,0.005652
100,No log,3.824266,0.004000
150,No log,3.690163,0.023198
200,No log,3.596719,0.045473
250,No log,3.528516,0.046888
300,No log,3.485589,0.070892
350,No log,3.471101,0.083143
400,No log,3.449289,0.114622
450,No log,3.454987,0.110869
500,No log,3.507018,0.102911


TrainOutput(global_step=1200, training_loss=2.403617960611979, metrics={'train_runtime': 647.5613, 'train_samples_per_second': 44.088, 'train_steps_per_second': 1.853, 'total_flos': 1645357984011324.0, 'train_loss': 2.403617960611979, 'epoch': 50.0})

In [62]:
from tqdm import tqdm


y_pred = []
y_true = tokenized_dataset["test"]["labels"]
with torch.no_grad():
    for i in tqdm(range(len(tokenized_dataset["test"]))):
        logits = model(
            **tokenizer(
                tokenized_dataset["test"][i]["text"],
                max_length=512,
                truncation=True,
                return_tensors="pt",
            ).to(device)
        )

        predicted_class_id = logits.logits.argmax().item()
        # model.config.id2label[predicted_class_id]

        y_pred.append(predicted_class_id)

100%|██████████| 308/308 [00:08<00:00, 34.77it/s]


In [67]:
print(f1_score(y_true, y_pred, average="macro",zero_division=0))
print(recall_score(y_true, y_pred, average="macro",zero_division=0))
print(precision_score(y_true, y_pred, average="macro",zero_division=0))

print(classification_report(y_true, y_pred,zero_division=0))


0.12125359461425037
0.12996031746031747
0.15628882841277694
              precision    recall  f1-score   support

           0       0.16      0.39      0.23        18
           1       0.50      0.67      0.57         3
           2       0.00      0.00      0.00         5
           3       0.00      0.00      0.00         6
           4       0.00      0.00      0.00         6
           5       0.00      0.00      0.00         6
           6       0.00      0.00      0.00         6
           7       0.00      0.00      0.00         5
           8       0.00      0.00      0.00         6
           9       0.00      0.00      0.00         6
          10       0.00      0.00      0.00         4
          11       0.00      0.00      0.00         5
          12       0.00      0.00      0.00         2
          13       0.00      0.00      0.00         1
          14       1.00      0.33      0.50         6
          15       1.00      0.67      0.80         3
          16       0.

По самым распространненым класам метрики (приемлемые):


                   precision recall  f1-score   support

          

          36       0.30      1.00      0.46        42
         
         
          42       0.75      0.60      0.67        10

